# Simple EDB Example

**STEPS**

1. Connect to the database
2. Get 'base' for configuration
3. Get components
4. Get reactions
5. Generate IDAES config

## 1. Connect to the database
By default, the class will attempt to connect to a MongoDB server running on port 27017 (default MongoDB port)
on the local host. You can pass a connection string to choose any other valid MongoDB instance.

In [2]:
from watertap.edb import ElectrolyteDB

print(f"connecting to {ElectrolyteDB.DEFAULT_URL}")
db = ElectrolyteDB()

connecting to mongodb://localhost:27017


### Failure to connect example
If you try to connect to a database and it fails, there will be a logged error message and then the
constructor will raise an error. If you really need to defer connection until later, add `check_connection=False`
to the constructor arguments.

In [4]:
db2 = ElectrolyteDB("mongodb://some.other.host", check_connection=False)
print("Didn't check the connection, no exception yet. But now...")
db2 = ElectrolyteDB("mongodb://some.other.host")

Didn't check the connection, no exception yet. But now...


Cannot connect to MongoDB server: some.other.host:27017: [Errno 11001] getaddrinfo failed, Timeout: 5.0s, Topology Description: <TopologyDescription id: 6172dfde1242bbb89e1fc47a, topology_type: Single, servers: [<ServerDescription ('some.other.host', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('some.other.host:27017: [Errno 11001] getaddrinfo failed')>]>


ServerSelectionTimeoutError: some.other.host:27017: [Errno 11001] getaddrinfo failed, Timeout: 5.0s, Topology Description: <TopologyDescription id: 6172dfde1242bbb89e1fc47a, topology_type: Single, servers: [<ServerDescription ('some.other.host', 27017) server_type: Unknown, rtt: None, error=AutoReconnect('some.other.host:27017: [Errno 11001] getaddrinfo failed')>]>

## 2. Get a 'base' for the configuration
Either `thermo` or `water_reaction`, currently.

In [ ]:
wb = db.get_base("thermo")

## 3. Get components
You can get components explicitly or by giving a list of elements.

In [ ]:
element_list = ["H", "O"]

components = db.get_components(element_names=element_list)

### Add components to base configuration

In [ ]:
for comp in components:
    print(f"adding component {comp.name}")
    wb.add(comp)

## 4. Get reactions
You generally want to get all the reactions associated with the same set of components.
You can also fetch reactions explicitly by name by providing a `reaction_names` keyword argument.
For fetching by component, the flag `any_components` controls its behavior. From the docstring:

     any_components: If False, the default, only return reactions where
                     one side of the reaction has all components provided.
                     If true, return the (potentially larger) set of reactions where
                     any of the components listed are present.

In [ ]:
reactions = db.get_reactions(component_names=wb.component_names)

### Add reactions to base configuration

In [ ]:
for r in reactions:
    print(f"adding {r.reaction_type} reaction: {r.name}")
    # set a reaction order that is different from the stoichiometry
    print(f"- stoichiometry = {r._data['stoichiometry']}")
    r.set_reaction_order('Liq', ('H2O',), ('H_+', 'OH_-'))
    print(f"- reaction order = {r._data['parameter_data']['reaction_order']}")
    # mess with the 'type' of the reaction, if necessary
    r._data["type"] = "inherent"
    wb.add(r)

## 5. Generate the IDAES config dict
This is done 'magically' by accessing the `idaes_config` property of the base configuration object.

In [ ]:
config = wb.idaes_config

display(config)